In [14]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt 
import timm  # timm (PyTorch Image Models) library for accessing pre-trained models and utilities for image classification
import os  
from tqdm.notebook import tqdm  # tqdm for displaying progress bars in Jupyter notebooks
from torchvision import transforms as T, datasets
from helper import view_classify, show_image, show_grid, accuracy # Importing custom helper functions for visualization and calculating accuracy

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
from torch import nn 
import torch.nn.functional as F

# Creating a pre-trained model from timm (PyTorch Image Models)
model_efficientnet = timm.create_model('tf_efficientnet_b4_ns', pretrained=False)  # Load a pre-trained model specified in CFG

# Customizing the classifier part of the model
model_efficientnet.classifier = nn.Sequential(
    nn.Linear(in_features=1792, out_features=625),  # First linear layer
    nn.ReLU(),  # ReLU activation function
    nn.Dropout(p=0.3),  # Dropout layer with a dropout probability of 0.3
    nn.Linear(in_features=625, out_features=256),  # Second linear layer
    nn.ReLU(),  # ReLU activation function
    nn.Linear(in_features=256, out_features=2)  # Final linear layer with 2 outputs for binary classification
)

In [17]:
# Load DenseNet-201
model_densenet = timm.create_model('densenet201', pretrained=False)
# Adjust the classifier layer of DenseNet-201 according to your project
model_densenet.classifier = nn.Sequential(
    nn.Linear(1920, 625),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(625, 256),
    nn.ReLU(),
    nn.Linear(256, 2)  # Assuming binary classification
)

In [18]:
# Creating a pre-trained model from timm (PyTorch Image Models)
model_vgg = timm.create_model('vgg11', pretrained=False)  # Load a pre-trained model specified in CFG
model_vgg.head.fc = nn.Linear(in_features=4096, out_features=2)

In [19]:
# Load the trained weights
model_efficientnet.load_state_dict(torch.load('PneumoniaModel.pth'))
model_densenet.load_state_dict(torch.load('DenseNetPneumoniaModel.pth'))
#model_vgg.load_state_dict(torch.load('VGG11PneumoniaModel.pth'))

<All keys matched successfully>

In [20]:
# Set both models to evaluation mode
model_efficientnet.to(device).eval()
model_densenet.to(device).eval()
model_vgg.to(device).eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [21]:
# Define transformations for the test dataset
test_transform = T.Compose([
    T.Resize(size=(224, 224)),  # Resize images to the specified size
    T.ToTensor(),  # Convert images to PyTorch tensors
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize
])

In [22]:
from torch.utils.data import DataLoader 

test_dataset = datasets.ImageFolder(root='./chest_xray_data/test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # Adjust batch_size as needed

In [23]:
# Soft Voting
def soft_voting_with_labels(model1, model2, model3, loader):
    model1.to(device).eval()
    model2.to(device).eval()
    model3.to(device).eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs1 = torch.softmax(model1(images), dim=1)
            outputs2 = torch.softmax(model2(images), dim=1)
            outputs3 = torch.softmax(model3(images), dim=1)

            avg_outputs = (outputs1 + outputs2 + outputs3) / 3
            _, predicted_classes = torch.max(avg_outputs, 1)

            all_predictions.extend(predicted_classes.cpu().numpy())
            all_labels.extend(labels.numpy())

    return all_predictions, all_labels


In [24]:
# Get predictions
predictions, true_labels = soft_voting_with_labels(model_efficientnet, model_densenet, model_vgg, test_loader)

In [25]:
accuracy = np.mean(np.array(predictions) == np.array(true_labels))
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 91.99%


In [26]:
del model_vgg, model_densenet, model_efficientnet
torch.cuda.empty_cache()  # Clear GPU cache